# SW중심대학 디지털 경진대회_SW와 생성AI의 만남 : AI부문
 - 이 AI 경진대회에서는 5초 분량의 오디오 샘플에서 진짜 사람 목소리와 AI가 생성한 가짜 목소리를 정확하게 구분할 수 있는 모델을 개발하는 것이 목표입니다.
 - 이 작업은 보안, 사기 감지 및 오디오 처리 기술 향상 등 다양한 분야에서 매우 중요합니다.
 - 제공되는 베이스라인 코드에서는 음성 파일에서 MFCC(Mel Frequency Cepstral Coefficients)로 특징을 추출하여 MLP(다층 퍼셉트론) 모델을 학습시키고 추론하는 과정을 포함하고 있으며, MFCC가 아닌 다른 방법론 역시 충분히 적용해볼 수 있습니다.

본 베이스라인은 다음과 같은 프로세스를 따릅니다.

<img src="https://r2-images.dacon.work/project/297/content/%EA%B7%B8%EB%A6%BC1.png">


1. 필요한 라이브러리 및 모듈 불러오기
librosa, sklearn, torch 등 다양한 라이브러리를 불러와 모델 학습에 필요한 기능들을 사용합니다.

2. 환경 설정 및 시드 고정
Config 클래스를 정의하여 모델 학습에 필요한 다양한 설정 값을 정의합니다.
예를 들어, 샘플링 레이트, 배치 사이즈, 학습률 등이 포함됩니다.
seed_everything 함수를 통해 모든 라이브러리의 시드를 고정하여 실험의 재현성을 보장합니다.

3. 데이터 불러오기 및 분할
pandas를 사용하여 CSV 파일로부터 학습 데이터를 불러옵니다.
train_test_split 함수를 사용하여 데이터를 학습 데이터셋과 검증 데이터셋으로 분할합니다.

4. 데이터셋 클래스 정의하기
CunstomDataset 클래스를 정의하여 학습에 사용할 데이터에 대한 변환과 전처리를 수행합니다.

5. 모델 정의하기
모델 학습 및 추론에 사용할 MLP 클래스를 정의합니다.

6. 모델 학습
선언한 MLP 클래스를 불러와 학습 데이터를 통해 모델을 학습시킵니다.
학습 과정에서 손실 함수를 정의하고, 옵티마이저를 설정하여 모델 파라미터를 최적화합니다.

7. 모델 추론 및 제출
학습된 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.
예측 결과를 CSV 파일로 저장하여 데이콘의 리더보드 시스템에 제출할 수 있는 형태로 구성합니다.

## Imports
모델 학습 및 추론에 사용할 라이브러리들을 불러옵니다.

1. Pandas(pandas)
    - 데이터 조작 및 분석을 위한 강력한 데이터 구조(예: DataFrame)를 제공합니다.
    - 다양한 데이터 포맷의 읽기 및 쓰기를 지원합니다.(예:CSV,Excel)
    - 데이터 클리닝, 탐색, 필터링, 집계 등의 작업에 이상적입니다.

2. NumPy(numpy)
    - 고성능의 다차원 배열 객체와 이를 다루기 위한 다양한 도구를 제공합니다.
    - 수학적, 과학적 연산을 위한 광범위한 함수를 포함합니다.
    - 데이터 전처리, 배열 변환 등에 널리 사용됩니다.

3. tqdm
    - 반복문에서 진행률을 표현해주는 라이브러리로, 코드 실행 상태를 쉽게 확인할 수 있습니다.

4. librosa
    - 오디오 및 음악 신호 분석을위한 python 패키지입니다.
    - 오디오 파일 로드, 특성 추출 등의 기능을 제공합니다.

5. os
    - os는 운영체제와 상호작용하기위한 다양한 기능을 제공합니다.
    - 파일 시스템을 탐색하고, 파일 및 디렉토리를 관리하며, 운영체제의 환경 변수를 접근하는 등의 작업을 수행할 수 있게 해줍니다.

##### PyTorch
- PyTorch는 오픈 소스 머신러닝 라이브러리로, 특히 딥러닝을 위해 사용됩니다.
- PyTorch의 구성 요소들은 모델을 설계하고, 학습하며 평가하는 과정에서 중요한 역할을 합니다.

1. torch
    - PyTorch의 기본 모듈로, 다차원 배열인 텐서(Tensor)를 조작하기 위한 다양한 함수와 도구들을 제공합니다.
    - 텐서는 PyTorch에서 데이터의 기본 단위이며, GPU를 사용한 연산 가속도를 지원합니다.

2. torch.nn
    - 신경망 모듈로 딥러닝 몯레을 구축하는 데 필요한 다양한 레이어(Layer)와 함수를 포함합니다.
    - 예를 들어, Linear,Conv2d,ReLU등의 클래스가 있으며, 이들은 모델의 구조를 정의하는데 사용됩니다.

3. torch.nn.functional
    - torch.nn 모듈의 함수 버전을 제공합니다.
    - 이 모듈은 활성화 함수(ReLU,sigmoid,tanh 등), 손실함수(cross_entropy, BCE_Loss 등)와 같은 다양한 연산을 함수 형태로 제공합니다.
    - 직접 레이어를 정의할 때나, 더 저수준의 커스텀 연산이 필요할 때 유용합니다.

4. torch.utils.data
    - 데이터 로딩 및 전처리를 위한 유틸리티를 제공합니다.
    - Dataset은 데이터셋을 정의하는 데 사용됩니다.
    - DataLoader는 데이터셋을 배치 단위로 불러오고, 이터레이션하는 데 사용됩니다.

5. torchmetircs
    - PyTorch 모델의 성능을 평가하기 위해 다양한 메트릭을 제공하는 라이브러리입니다.
    - 다양한 평가 지표를 제공하여 반복적인 성능 평가 코드를 간소화하고, 다양한 메트릭을 일관된 방식으로 계산할 수 있도록 돕습니다.

In [1]:
import os
import random

import torch
import torchaudio

import librosa
import numpy as np
import pandas as pd

from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device -", device)

Using device - cuda


## Config
- 딥러닝 모델을 학습하기 전에 설정해야하는 다양한 매개변수를 정의하는 설정 클래스입니다.
- 클래스를 사용하여 학습에 필요한 설정 값을 미리 지정합니다.
- 이러한 방식은 학습 파라미터를 일관되고 체계적으로 관리할 수 있게 해주며, 실험의 재현성을 높이는 데 도움이 됩니다.

##### 오디오 신호
- 우리가 듣는 소리는 공기의 압력 변화로, 이것을 디지털 신호로 변환한 것이 오디오 신호입니다.
- 이 신호는 시간에 따라 변하는 진폭 값을 가지고 있습니다.


1. SR(Sample Rate)
    - 오디오 데이터의 샘플링 레이트를 설정합니다.
        - 높은 샘플링 레이트는 더 높은 주파수의 소리를 캡처할 수 있지만, 처리에 더 많은 계산 자원이 필요합니다.
    - 오디오 데이터의 초당 샘플 수를 정의합니다.

2. N_MFCC(Number of MFCCs)
    - N_MFCC는 멜 주파수 켑스트럼 계수(MFCCs)의 개수를 의미합니다.
    - MFCC는 오디오 신호의 주파수 특성을 인간의 청각 특성에 맞게 변환한 것으로, 음성 인식 등의 분야에서 많이 사용됩니다.

3. ROOT_FOLDER
    - 데이터셋의 루트 폴더 경로를 설정합니다.

4. N_CLASSES
    - 분류할 클래스의 수를 설정합니다.
    - 모델의 출력 차원을 설정할 때 사용됩니다.

5. BATCH_SIZE
    - 배치 크기를 설정합니다.
    - 학습 시 한 번에 처리할 데이터 샘플의 수를 정의합니다
    - 큰 배치 크기는 메모리 사용량을 증가시키지만, 학습 속도를 높입니다.

6. N_EPOCHS
    - 학습 에폭 수를 설정합니다.
    - 전체 데이터셋을 학습할 횟수를 정의합니다.
    - 에폭 수가 너무 적으면 과소적합이 발생할 수 있고, 너무 많으면 과적합이 발생할 수 있습니다.

7. LR (Learning Rate)
    - 학습률을 설정합니다.
    - 모델의 가중치를 업데이트할 때 사용되는 학습 속도를 정의합니다.
    - 학습률이 너무 크면 학습이 불안정해질 수 있고, 너무 작으면 학습 속도가 느려집니다.

In [4]:
class Config:
    SR = 32000
    N_MFCC = 13
    # Dataset
    ROOT_FOLDER = os.path.join(".", "data")
    # Training
    N_CLASSES = 2
    BATCH_SIZE = 96
    N_EPOCHS = 5
    LR = 3e-4
    # Others
    SEED = 42
    
CONFIG = Config()

8. SEED
    - 재현성을 위해 SEED값을 고정하는 SEED를 설정해줍니다.

In [5]:
def seed_everything(seed):
    """ Fixed RandomSeed
    아래의 코드는 머신러닝이나 딥러닝 모델을 훈련할 때, 결과의 재현성을 보장하기 위해 사용되는 함수입니다.
    이 함수는 다양한 랜덤 시드를 고정하여, 실행할 때마다 동일한 결과를 얻기 위해 사용됩니다.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CONFIG.SEED)  # Seed 고정

## Data Pre-processing : MFCC

여기서는 인공지능 모델 학습에서 중요한 데이터 전처리를 수행합니다.

- MFCC(Mel Frequency Cepstral Coefficients)는 음성 신호의 주파수 특성을 인간의 청각 특성에 맞춰 변환한 것으로, 음성 인식 등의 음성 처리 응용 분야에서 널리 사용되는 특징 추출 방법입니다.
- MFCC는 주파수 도메인에서 신호의 중요한 정보를 캡처하여 입력 데이터를 효과적으로 표현합니다.

장점
- 멜 스케일과 로그 변환을 통해 인간의 청각 특성을 잘 반영합니다.
- 고주파 잡음에 상대적으로 덜 민감합니다.

단점
- 고정된 크기의 윈도우를 사용하기 때문에 음성 신호의 비정상성을 잘 포착하지 못할 수 있습니다.

#### 프레임 분할 (Framing)
1. 프레임 분할
    - 음성 신호는 시간에 따라 변하므로, 신호를 짧은 구간으로 나누어 각 구간에서 분석을 수행합니다.
    - 일반적으로 20-40ms 길이의 프레임으로 나누며, 프레임 간 겹침을 포함하는 경우가 많습니다.

2. 윈도우 함수 적용
    - 각 프레임에 윈도우 함수를 적용하여 신호의 경계 효과를 줄입니다.

3. Window Function
    - 오디오 신호를 작은 구간으로 나눌 때, 구간의 경계(양 끝값)이 불연속하게 끊기게 되어 실제 신호와 달라지는 문제가 발생합니다.
    - 이렇게 windowing이 신호 특성에 주는 영향을 최소화하기 위해 양 경계값을 0으로 수렴하도록하여 경계 효과를 줄입니다.

4. 멜 스케일(Mel Scale)
    - 인간의 청각 특성을 반영한 주파수 척도입니다.
    - 인간은 낮은 주파수에서 더 작은 차이를 느끼고, 높은 주파수에서 더 큰 차이를 느낍니다.
    - 멜 스케일은 이러한 특성을 반영하여 주파수를 비선형적으로 변환합니다.

- 이 코드는 MRCC 특징을 추출하고, 이를 학습에 사용할 형식으로 변환하는 함수를 정의하는 코드입니다.
- `librosa.load`를 사용하여 `row['path']`에 해당하는 오디오 파일을 로드합니다.
- 샘플링 레이트는 CONFIG.SR로 지정됩니다.
- `librosa.feature.mfcc`를 사용하여 오디오 신호 y로부터 MFCC 특징을 추출합니다.
- `CONFIG.N_MFCC`는 추출할 MFCC 계수의 개수를 지정합니다.
- 추출된 MFCC는 프레임별로 계산되므로, 각 프레임의 평균값을 구하여 전체 파일에 대한 MFCC 특징을 대표하는 벡터를 얻습니다.
- `train_mode = True`인 경우, 현재 행의 레이블을 읽어와 CONFIG.N_CLASSES 길이의 벡터로 변환합니다.
- 레이블이 'fake'이면 첫 번째 원소를 1로, 'real'이면 두 번째 원소를 1로 설정합니다.
- 이 벡터를 labels 리스트에 추가합니다.

In [6]:
from librosa import feature

In [7]:
def get_mfcc_feature(df, root="./data", train_mode=True):
    features = []
    labels = []
    for _, row in tqdm(df.iterrows()):
        # librosa패키지를 사용하여 wav 파일 load
        audio_pth = os.path.join(root, *row['path'][1:].split("/"))
        y, sr = librosa.load(audio_pth, sr=CONFIG.SR)
        
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        mfcc = np.mean(mfcc.T, axis=0)
        features.append(mfcc)

        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)

    if train_mode:
        return features, labels
    else:
        return features, None

## Dataset

#### VoiceDataset
- VoiceDataset 클래스는 PyTorch의 Dataset 클래스를 상속받아 데이터 로딩과 관련된 과정을 효율적으로 관리할 수 있도록 정의된 클래스입니다.
- 이 클래스는 MFCC 특징과 레이블 데이터를 다루며, 딥러닝 모델 학습에 적합한 형식으로 데이터를 제공하는 역할을 합니다.
- 주요 구성 요소는 다음과 같습니다.

1. __init__
- 데이터셋을 초기화할 때 필요한 매개변수를 설정합니다.
- mfcc는 오디오 파일에서 추출한 MFCC 특징들의 리스트입니다.
- label은 해당 오디오 파일의 레이블 리스트로, 레이블이 없는 경우 None이 될 수 있습니다.

2. __len__
- 데이터셋의 총 데이터 개수를 반환합니다.
- 주어진 MFCC 리스트의 길이와 동일합니다.

3. __getitem__
- 주어진 인덱스에 해당하는 데이터를 데이터셋에서 불러와 반환합니다.
- 레이블이 있는 경우 MFCC와 레이블을 함께 반환하고, 없는 경우 MFCC만 반환합니다.

In [11]:
from torchvision.datasets.utils import download_and_extract_archive


class VoiceDataset(Dataset):
    download_url = "https://drive.usercontent.google.com/download?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf&export=download&authuser=0&confirm=t&uuid=c40c278b-d74b-4b75-bc79-09e8a3ccffa4&at=APZUnTUvIVFVM9gjGNUCmDb4YZCy%3A1719807236671"
    download_root = None
    
    @classmethod
    def download(cls, root='./data', filename="download.zip", md5=None):
        cls.download_root = root
        filepath = os.path.join(root, filename)
        if not os.path.exists(filepath):
            download_and_extract_archive(cls.download_url, root, root, filename, md5)
            print("Extraction completed.")
        else:
            print(f"File already exists in {filepath}")
    
    @classmethod
    @property
    def trainset_path(cls):
        if cls.download_root is None:
            cls.download()
        
        return os.path.join(cls.download_root, "train.csv")

    @classmethod
    @property
    def testset_path(cls):
        if cls.download_root is None:
            cls.download()

        return os.path.join(cls.download_root, "test.csv")

    @classmethod
    @property
    def submission_form_path(cls):
        if cls.download_root is None:
            cls.download()

        return os.path.join(cls.download_root, "sample_submission.csv")
    
    def __init__(self, dataframe, root="./data", train=True):
        self.mfcc, self.label = get_mfcc_feature(dataframe, root, train)

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]

#### Train & Validation Split
여기서는 인공지능 모델 학습에서 중요한 데이터 전처리를 수행합니다.

모델을 훈련하기 전에, 전체 데이터 세트를 두 개의 서브셋으로 나눠줍니다.
하나는 모델 학습에 사용되는 학습 데이터 세트이고, 다른 하나는 학습된 모델의 성능을 평가하기 위한 검증 데이터 세트입니다.

이렇게 데이터를 분할하는 이유는 모델이 새로운 데이터에 대해 얼마나 잘 일반화하는지 평가하기 위해서입니다.
모델이 훈련 데이터에만 과도하게 최적화되지 않도록 검증 데이터 세트를 사용하여 모델의 성능을 주기적으로 검증합니다.

In [12]:
from sklearn.model_selection import train_test_split as train_valid_split

VoiceDataset.download(root=CONFIG.ROOT_FOLDER)

df = pd.read_csv(VoiceDataset.trainset_path)
train_df, valid_df, _, _ = train_valid_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED)

train_dataset = VoiceDataset(train_df, root=CONFIG.ROOT_FOLDER, train=True)
val_dataset = VoiceDataset(valid_df, root=CONFIG.ROOT_FOLDER, train=True)

File already exists in .\data\download.zip


44350it [12:20, 59.86it/s]
11088it [02:55, 63.09it/s]


## DataLoader

- DataLoader는 구축된 데이터셋에서 배치크기(batch_size)에 맞게 데이터를 추출하고, 필요에 따라 섞거나(shuffle=True) 순서대로 반환(shuffle=False)하는 역할을 합니다.
- 훈련 데이터(train_loader)는 일반적으로 섞어서 모델이 데이터에 덜 편향되게 학습하도록하며, 검증 데이터(val_loader)는 모델 성능 평가를 위해 순서대로 사용하고, 테스트 데이터(test)loader)는 최종적인 추론을 위해 사용합니다.

이렇게 DataLoader를 사용함으로써, 효율적인 데이터 처리와 모델 학습 및 평가가 가능해집니다.

In [13]:
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

## Define Model

##### MLP
- MLP모델은 여러 층의 노드로 구성되며 각 노드는 입력값과 가중치를 통해 계산된 값을 출력으로 전달합니다.
- 일반적으로 입력층, 은닉층(hidden Layer), 출력층으로 구성됩니다.
- 은닉층이 하나 이상 존재하며, 비선형 활성화 함수를 사용하여 입력 데이터의 복잡한 패턴을 학습할 수 있습니다.
- MLP는 입력과 출력 사이의 복잡한 관계를 모델링하는 데 매우 효과적이며, 회귀 분석, 분류 문제 등 다양한 기계 학습 작업에 사용됩니다.

1. 입력층(Input Layer)
    - 모델에 데이터를 전달합니다.
    - 입력층의 노드 수는 입력 데이터의 특징 수와 동일합니다.
    - 본 베이스라인에서는 MFCC의 개수를 입력으로 사용합니다.

2. 은닉층(Hidden Layer)
    - 각 은닉층의 노드는 이전 층의 출력을 입력으로 받아 계산을 수행합니다.
    - 활성화 함수는 노드의 출력 값을 비선형으로 변환하여 다음 층으로 전달합니다.
    - 본 베이스라인에서는 입력 차원을 은닉 차원으로 변환하고 활성화 함수를 사용하여 비선형성을 추가해 모델의 표현력을 높입니다.

3. 출력층(Output Layer)
    - 출력층은 모델의 최종 예측 값을 생성합니다.
    - 본 베이스라인의 출력 차원은 CONFIG.N_CALSSES가 출력차원입니다.
    - MLP 모델은 기본적인 신경망 구조로, 다양한 문제에 적용할 수 있습니다.

참가자 여러분들께서는 MLP모델을 통해 기본적인 신경망에 대해 학습하고, 다양한 모델을 적용할 수 있습니다.

<img src="https://r2-images.dacon.work/project/297/content/%EA%B7%B8%EB%A6%BC4.png">

In [14]:
class MLP(nn.Module):
    def __init__(self, input_dim=CONFIG.N_MFCC, hidden_dim=128, output_dim=CONFIG.N_CLASSES):
        """
        모델의 각 구성요소를 초기화 합니다.
        :param input_dim: MFCC의 개수를 의미합니다.
        :param hidden_dim: 은닉층의 차원 수입니다.
        :param output_dim: 분류할 클래스의 수를 의미합니다.
        """
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        입력 데이터를 순차적으로 세 개의 선형 계층과 ReLU 활성화 함수를 거쳐 최종적으로 시그모이드 함수를 적용하여 출력 확률을 계산합니다
        """
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

## Train & Validation
아래의 코드는 PyTorch를 사용한 딥러닝 모델의 훈련 및 검증 과정과 다중 레이블 AUC 점수 계산을 포함합니다.

In [15]:
from sklearn.metrics import roc_auc_score

def train(model, optimizer, train_loader, val_loader, device):
    """
    1. 모델 및 Loss 함수 설정
        - 모델을 주어진 device(GPU 또는 CPU)로 이동하고, BCELoss(Binary Cross Entropy)를 Loss 함수로 설정합니다.
    2. 최적의 모델 저장을 위한 변수 초기화
        - 학습 과정동안 가장 좋은 검증 점수를 가진 모델을 추적하기 위해 최적의 점수(best_val_score)와 모델(best_model)을 저장할 변수를 초기화합니다.
    3.에포크별 훈련
        - 설정된 에포크 수(CONFIG.N_EPOCHS)만큼 반복하여 모델을 훈련합니다.
        - train_loader에서 배치별로 스펙트로그램과 레이블을 불러옵니다.
        - 모델을 통해 예측을 수행하고 , 손실을 계산합니다.
        - loss.backward()를 수행하고, 파라미터를 업데이트 합니다.
        - 이후, 훈련 손실을 기록합니다.
    4. 검증 및 최적 모델 업데이트
        - 각 에포크 후에 검증 함수를 호출해 모델의 성능을 평가하며, 현재 에포크에서 더 나은 검증 점수를 달성하면 이 모델을 최적의 모델로 간주하고 저장합니다.
    """
    
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}]')
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [16]:
def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [17]:
def validation(model, criterion, val_loader, device):
    """
    1. 모델 평가 모드 설정
        - model.eval()을 호출하여 모델을 평가 모드로 설정합니다.
    2. 손실 계산 및 예측 수집
        - val_loader에서 배치별로 스펙트로그램과 레이블을 불러와, 모델을 통해 예측을 수행합니다.
        - 손실 값을 기록하고 예측된 확률과 실제 레이블을 수집합니다.
    3. 성능 평가
        - 수집된 모든 예측과 실제 레이블을 사용해 multiLabel_AUC 함수를 호출하여 평균 AUC 점수를 계산합니다.
    """
    
    model.eval()
    val_loss, all_labels, all_probs = [], [], []

    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)

            probs = model(features)

            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())

        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)

        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)

    return _val_loss, auc_score

### Run
모델을 초기화하고, 옵티마이저를 설정한 다음 모델 훈련 함수를 호출하여 실제로 훈련 과정을 실행하는 단계입니다.

In [18]:
model = MLP()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CONFIG.LR)

In [ ]:
infer_model = train(model, optimizer, train_loader, val_loader, device)

### Inference
테스트 데이터셋에 대한 추론은 다음 순서로 진행됩니다.

1. test 데이터 준비
    - 앞서 정의한 VoiceDataset 클래스와 DataLoader를 사용하여 추론에 사용할 테스트 데이터를 설정합니다.
2. 모델 및 디바이스 설정
    - 모델을 주어진 device(GPU 또는 CPU)로 이동시키고, 평가모드로 전환합니다.
3. 예측 수행
    - 예측 결과를 저장한 빈 리스트를 초기화하고 test_loader에서 배치별로 데이터를 불러와 예측을 수행합니다.
    - 각 배치에 대해 스펙트로그램 데이터를 device로 이동시킵니다.
    - 모델 예측 확률(probs)을 계산합니다.
    - 예측 확률을 predictions리스트에 추가합니다.

In [ ]:
test_df = pd.read_csv(VoiceDataset.testset_path)
test_dataset = VoiceDataset(test_df, CONFIG.ROOT_FOLDER, False)
test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [ ]:
predicted_labels = inference(infer_model, test_loader, device)

### Submission
추론 결과를 제출 양식에 덮어 씌워 CSV 파일로 생성하는 과정은 다음과 같습니다.

1. 제출 양식 로드
    - pd.read_csv('./sample_submission.csv')를 사용하여 제출을 위한 샘플 형식 파일을 로드합니다.
    - 이 파일은 일반적으로 각 테스트 샘플에 대한 ID와 예측해야 하는 필드가 포함된 템플릿 형태를 가지고 있습니다.
2. 예측 결과 할당
    - submit.iloc[:,1:] = preds 추론함수(inference)에서 반환된 예측결과(preds)를 샘플 제출 파일에 2번째 열부터 할당합니다.
3. 제출 파일 저장
    - 수정된 제출 파일을 baseline_submit 이란 이름의 CSV 파일로 저장합니다.
    - index=False는 파일 저장시 추가적인 index가 발생하지 않도록 설정하여, 제작한 제출 파일과 동일한 형태의 파일을 저장합니다.

In [ ]:
submit = pd.read_csv(VoiceDataset.submission_form_path)
submit.iloc[:, 1:] = predicted_labels
submit.head()

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)